#### Adicionando o caminho do projeto ao path

In [1]:
import os
os.chdir('/Users/edithvidal/Documents/Github/data_science_analysis/')

#### Instalando sklearn

In [2]:
%pip install scikit-learn
%pip install category_encoders
%pip install imbalanced-learn



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Importando as bibliotecas necessárias

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay


#### Lendo dados da base após análise

In [4]:
df_accidents = pd.read_csv(
    "analysis/datatran2018_2024_v3.csv",
    parse_dates=["mes_ano"]
)
df_accidents.info()
df_accidents.reset_index(drop=True, inplace=True)

/var/folders/z1/3syxl8gd5ks431s8c9n9mrs00000gn/T/ipykernel_2258/2452157731.py:1: DtypeWarning: Columns (0,2,3,4,5,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accidents = pd.read_csv(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467078 entries, 0 to 467077
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   tipo_data               429733 non-null  object        
 1   mes_ano                 467078 non-null  datetime64[ns]
 2   dia_semana              429733 non-null  object        
 3   fase_dia                429733 non-null  object        
 4   condicao_metereologica  429733 non-null  object        
 5   uf                      429733 non-null  object        
 6   br                      429733 non-null  float64       
 7   uso_solo                429733 non-null  object        
 8   tipo_pista              429733 non-null  object        
 9   sentido_via             429733 non-null  object        
 10  tipo_acidente           429733 non-null  object        
 11  causa_acidente          429733 non-null  object        
 12  classificacao_acidente  429733

#### Removendo colunas que não serão utilizadas para a classificação

In [5]:
df = df_accidents.copy()
df.drop(columns=[
    "uf",
    "mes_ano",
    "tipo_data",
    "classificacao_acidente",
    "pessoas",
    "veiculos"
], inplace=True)

df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429733 entries, 0 to 429732
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   dia_semana              429733 non-null  object 
 1   fase_dia                429733 non-null  object 
 2   condicao_metereologica  429733 non-null  object 
 3   br                      429733 non-null  float64
 4   uso_solo                429733 non-null  object 
 5   tipo_pista              429733 non-null  object 
 6   sentido_via             429733 non-null  object 
 7   tipo_acidente           429733 non-null  object 
 8   causa_acidente          429733 non-null  object 
dtypes: float64(1), object(8)
memory usage: 32.8+ MB


#### Agrupando tipos de acidente para facilitar a classificação

In [6]:
df["tipo_acidente"].value_counts()

tipo_acidente
Colisão traseira                   81314
Saída de leito carroçável          67518
Colisão transversal                54276
Tombamento                         35660
Colisão frontal                    28970
Colisão lateral                    24476
Colisão lateral mesmo sentido      23272
Atropelamento de pedestre          19884
Queda de ocupante de veículo       19182
Colisão com objeto                 16729
Colisão com objeto estático        14144
Capotamento                        12119
Incêndio                            8092
Atropelamento de animal             7337
Engavetamento                       6732
Colisão lateral sentido oposto      6444
Derramamento de carga               1032
Eventos atípicos                    1012
Colisão com objeto em movimento      941
Danos eventuais                      599
Name: count, dtype: int64

In [7]:
accidents = {
    "Colisão traseira": "Colisão",
    "Colisão transversal": "Colisão",
    "Colisão frontal": "Colisão",
    "Colisão lateral": "Colisão",
    "Colisão lateral mesmo sentido": "Colisão",
    "Colisão lateral sentido oposto": "Colisão",
    "Colisão com objeto": "Colisão",
    "Colisão com objeto estático": "Colisão",
    "Colisão com objeto em movimento": "Colisão",
    "Atropelamento de animal": "Atropelamento",
    "Atropelamento de pedestre": "Atropelamento",
    "Capotamento": "Perda de controle",
    "Tombamento": "Perda de controle",
    "Saída de leito carroçável": "Perda de controle",
    "Engavetamento": "Perda de controle",
    "Derramamento de carga": "Evento externo",
    "Incêndio": "Evento externo",
    "Eventos atípicos": "Evento externo",
    "Danos eventuais": "Evento externo",
    "Queda de ocupante de veículo": "Queda de ocupante de veículo"
}

df["tipo_acidente"] = df["tipo_acidente"].map(accidents)
df["tipo_acidente"].value_counts()

tipo_acidente
Colisão                         250566
Perda de controle               122029
Atropelamento                    27221
Queda de ocupante de veículo     19182
Evento externo                   10735
Name: count, dtype: int64

#### OneHotEncoder + RandomForestClassifier

In [13]:
target_col = 'tipo_acidente'

# 1. Features categóricas que você quer usar
categorical_features = [
    'dia_semana', 'fase_dia', 'condicao_metereologica',
    'uso_solo', 'tipo_pista', 'sentido_via', 'causa_acidente'
]

# 2. Separar X e y
X = df[categorical_features]
y = df[target_col]

# 3. Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# 4. Preprocessamento (OneHotEncoder para dados categóricos)
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# 5. Montar o pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        class_weight='balanced',  # trata desbalanceamento
        random_state=42,
        n_estimators=30,
        max_depth=10,
        n_jobs=-1  # usa todos os núcleos disponíveis
    ))
])

# 6. Treinar o modelo
pipeline.fit(X_train, y_train)

# 7. Avaliar
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

                              precision    recall  f1-score   support

               Atropelamento       0.86      0.73      0.79      8166
                     Colisão       0.82      0.59      0.69     75170
              Evento externo       0.36      0.79      0.49      3220
           Perda de controle       0.47      0.65      0.54     36609
Queda de ocupante de veículo       0.12      0.19      0.15      5755

                    accuracy                           0.60    128920
                   macro avg       0.52      0.59      0.53    128920
                weighted avg       0.68      0.60      0.62    128920



In [ ]:
# Gerar matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_)

# Visualizar
fig, ax = plt.subplots(figsize=(10, 8))
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
plt.title('Matriz de Confusão - RandomForest + OneHotEncoder')
plt.tight_layout()
plt.show()

#### TargetEncoder + RandomForestClassifier

In [8]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

# Variáveis
target_col = 'tipo_acidente'
categorical_features = [
    'dia_semana',
    'fase_dia',
    'condicao_metereologica',
    'uso_solo',
    'tipo_pista',
    'sentido_via',
    'causa_acidente'
]

# Separar dados
X = df[categorical_features]
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Pipeline com TargetEncoder + RandomForest
pipeline = Pipeline([
    ('encoder', TargetEncoder(cols=categorical_features)),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])

# Grade de hiperparâmetros para RandomForest
param_distributions = {
    'clf__n_estimators': randint(50, 100),          # de 50 até 149
    'clf__max_depth': randint(5, 10),               # de 5 até 19
    'clf__min_samples_split': randint(2, 11),       # de 2 até 10
    'clf__min_samples_leaf': randint(1, 5),         # de 1 até 4
    'clf__max_features': ['sqrt', 'log2']           # evita 'auto' (depreciado no RF)
}


random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='f1_macro',
    n_jobs=2,
    random_state=42,
    verbose=1
)

# Executa busca
random_search.fit(X_train, y_train)

# Melhores parâmetros
print("Melhores parâmetros:", random_search.best_params_)
print("Melhor score (f1_macro):", random_search.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Melhores parâmetros: {'clf__max_depth': 9, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 3, 'clf__min_samples_split': 6, 'clf__n_estimators': 73}
Melhor score (f1_macro): 0.5329068990347604


In [ ]:
# Variáveis
target_col = 'tipo_acidente'
categorical_features = [
    'dia_semana', 
    'fase_dia', 
    'condicao_metereologica',
    'uso_solo', 
    'tipo_pista', 
    'sentido_via', 
    'causa_acidente'
]

# Separar dados
X = df[categorical_features]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Pipeline com TargetEncoder + RandomForest
pipeline = Pipeline([
    ('encoder', TargetEncoder(cols=categorical_features)),
    ('classifier', RandomForestClassifier(
        class_weight='balanced',
        n_estimators=73,
        n_jobs=-1,
        max_depth=9,
        min_samples_split=6,
        min_samples_leaf=3,
        max_features="log2",
        random_state=42
    ))
])

# Treinar
pipeline.fit(X_train, y_train)

# Previsões
y_pred = pipeline.predict(X_test)

# Relatório textual
print(classification_report(y_test, y_pred))


In [ ]:
# Gerar matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_)

# Visualizar
fig, ax = plt.subplots(figsize=(10, 8))
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
plt.title('Matriz de Confusão - RandomForest + TargetEncoder')
plt.tight_layout()
plt.show()


#### TargetEncoder + DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV


# Variáveis
target_col = 'tipo_acidente'
categorical_features = [
    'dia_semana', 
    'fase_dia', 
    'condicao_metereologica',
    'uso_solo', 
    'tipo_pista', 
    'sentido_via', 
    'causa_acidente'
]

# Separar dados
X = df[categorical_features]
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Pipeline
pipeline = Pipeline([
    ('encoder', TargetEncoder(cols=categorical_features)),
    ('clf', DecisionTreeClassifier(class_weight='balanced', random_state=42))
])

# Grade de hiperparâmetros
param_grid = {
    'clf__max_depth': [None, 5, 10, 15, 20],
    'clf__min_samples_split': [2, 5, 10, 20],
    'clf__min_samples_leaf': [1, 2, 5, 10]
}

# Configura o GridSearch
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,               # número de folds
    scoring='f1_macro',  # métrica para multiclasse balanceada
    n_jobs=-1,
    verbose=1
)

# Executa busca
grid_search.fit(X_train, y_train)

# Melhores parâmetros
print("Melhores parâmetros:", grid_search.best_params_)

In [ ]:
# Variáveis
target_col = 'tipo_acidente'
categorical_features = [
    'dia_semana', 'fase_dia', 'condicao_metereologica',
    'uf', 'uso_solo', 'tipo_pista', 'sentido_via', 'causa_acidente'
]

# Separar dados
X = df[categorical_features]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Pipeline com hiperparâmetros ajustados
pipeline = Pipeline([
    ('encoder', TargetEncoder(cols=categorical_features)),
    ('classifier', DecisionTreeClassifier(
        class_weight='balanced',
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=5,
        random_state=42
    ))
])

# Treinar modelo
pipeline.fit(X_train, y_train)

# Prever
y_pred = pipeline.predict(X_test)

# Avaliar
print(classification_report(y_test, y_pred))

In [ ]:
# Gerar matriz de confusão
cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_)

# Visualizar
fig, ax = plt.subplots(figsize=(10, 8))
disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
plt.title('Matriz de Confusão - RandomForest + TargetEncoder')
plt.tight_layout()
plt.show()


#### XGBoost

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
from sklearn.metrics import classification_report

# Variáveis
target_col = 'tipo_acidente'
categorical_features = [
    'dia_semana',
    'fase_dia',
    'condicao_metereologica',
    'uso_solo',
    'tipo_pista',
    'sentido_via',
    'causa_acidente'
]

# Separar dados
X = df[categorical_features]
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Pipeline com TargetEncoder + XGBClassifier
pipeline = Pipeline([
    ('encoder', TargetEncoder(cols=categorical_features)),
    ('clf', XGBClassifier(
        objective='multi:softmax',  # ou 'multi:softprob' para probabilidades
        num_class=len(y.unique()),  # obrigatório para classificação multiclasse
        eval_metric='mlogloss',
        use_label_encoder=False,
        n_jobs=-1,
        random_state=42
    ))
])

# Grade de hiperparâmetros para XGBoost
param_distributions = {
    'clf__n_estimators': randint(50, 150),
    'clf__max_depth': randint(3, 10),
    'clf__learning_rate': uniform(0.01, 0.3),
    'clf__subsample': uniform(0.6, 0.4),
    'clf__colsample_bytree': uniform(0.6, 0.4),
    'clf__gamma': uniform(0, 5)
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='f1_macro',
    n_jobs=2,
    random_state=42,
    verbose=1
)

# Executa busca
random_search.fit(X_train, y_train)

# Melhores parâmetros
print("Melhores parâmetros:", random_search.best_params_)
print("Melhor score (f1_macro):", random_search.best_score_)

# Avaliação no teste
y_pred = random_search.predict(X_test)
print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))
